# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, make_scorer
from deepod.metrics import tabular_metrics

from tqdm import tqdm
from time import time

In [2]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

#X_val = np.load('data/x_val.npy')
#y_val = np.load('data/y_val.npy')

In [3]:
#X_train = np.concatenate((X_train, X_val), axis=0)
#y_train = np.concatenate((y_train, y_val), axis=0)

# Playground

In [4]:
np.unique(y_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([ 7928,  8357, 10489,  8184, 11611,   682,   286, 19323,  7913,
         7186,  7679,  8258,  8198,  8006,  7656]))

7 is normal class

In [5]:
y_test_binary = np.where(y_test == 7, 'normal', 'anomaly')

In [6]:
normal_data = X_train[y_train == 7] # normal class

In [7]:
len(normal_data)

19323

# Helpers

In [8]:
def evaluate_model(model, y_test, y_pred):
    positive_class = "normal"
    
    accuracy = accuracy_score(y_test, y_pred)
    
    precision = precision_score(y_test, y_pred, pos_label=positive_class)
    recall = recall_score(y_test, y_pred, pos_label=positive_class)
    f1 = f1_score(y_test, y_pred, pos_label=positive_class)
    
    metrics = {
        'model': [model],
        'accuracy': [accuracy],
        'precision': [precision],
        'recall': [recall],
        'f1': [f1],
    }
    
    return metrics

# DIF

In [10]:
from deepod.models.tabular import DeepIsolationForest

In [11]:
clf_dif = DeepIsolationForest()
clf_dif.fit(normal_data, y=None)
y_preds_dif = clf_dif.predict(X_test)

Start Training...


100%|██████████| 50/50 [00:05<00:00,  9.45it/s]


Start Inference on the training data...
Start Inference...


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


Start Inference...


100%|██████████| 50/50 [00:56<00:00,  1.14s/it]


In [14]:
y_preds_dif_binary = np.where(y_preds_dif == 1, 'normal', 'anomaly')

In [21]:
results_dif = pd.DataFrame(evaluate_model("DIF", y_test_binary, y_preds_dif_binary))
results_dif

,model,accuracy,precision,recall,f1
0,DIF,0.156965,0.156965,1.0,0.271339


# SLAD

In [16]:
from deepod.models.tabular import SLAD

In [17]:
clf_slad = SLAD()
clf_slad.fit(normal_data, y=None)
y_preds_slad = clf_slad.predict(X_test)

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 50, ensemble size: 20
len pool: [ 1  4  5  6  7  8  9 10 11 13 14 16 17 18 19 20 23 26 28 29 31 32 34 35
 36 37 39 40 41 43 45 46 47 48 50 51 54 55 56 57 58 59 60 62 64 66 68 71
 73 74]
epoch  1, training loss: 0.728733, time: 1.5s
epoch 10, training loss: 0.695240, time: 0.8s
epoch 20, training loss: 0.695133, time: 0.8s
epoch 30, training loss: 0.695192, time: 0.8s
epoch 40, training loss: 0.695134, time: 0.9s
epoch 50, training loss: 0.695210, time: 0.8s
epoch 60, training loss: 0.695120, time: 0.9s
epoch 70, training loss: 0.695145, time: 0.9s
epoch 80, training loss: 0.695100, time: 0.8s
epoch 90, training loss: 0.695126, time: 0.9s
epoch100, training loss: 0.695124, time: 0.9s
Start Inference on the training data...


In [18]:
y_preds_slad_binary = np.where(y_preds_slad == 0, 'normal', 'anomaly')

In [19]:
results_slad = pd.DataFrame(evaluate_model("SLAD", y_test_binary, y_preds_slad_binary))
results_slad

,model,accuracy,precision,recall,f1
0,SLAD,0.983344,1.0,0.893889,0.943972


# ICL

In [9]:
from deepod.models.tabular import ICL

In [10]:
clf_icl = ICL(epochs=5)
clf_icl.fit(normal_data, y=None)
y_preds_icl = clf_icl.predict(X_test)

Start Training...
ensemble size: 2
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=64, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(65, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(65, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(65, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=

testing: 100%|██████████| 476/476 [00:00<00:00, 601.38it/s]


In [11]:
y_preds_icl_binary = np.where(y_preds_icl == 0, 'normal', 'anomaly')

In [12]:
results_icl = pd.DataFrame(evaluate_model("ICL", y_test_binary, y_preds_icl_binary))
results_icl

,model,accuracy,precision,recall,f1
0,ICL,0.984297,1.0,0.899958,0.947345


# NeuTraL

In [13]:
from deepod.models.tabular import NeuTraL

In [14]:
clf_neutral = NeuTraL(epochs=1)
clf_neutral.fit(normal_data, y=None)
y_preds_neutral = clf_neutral.predict(X_test)

Start Training...
ensemble size: 1
epoch  1, training loss: 0.134671, time: 106.3s
Start Inference on the training data...


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 10.75 GiB total capacity; 9.35 GiB already allocated; 2.69 MiB free; 9.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
y_preds_neutral_binary = np.where(y_preds_neutral == 0, 'normal', 'anomaly')

In [ ]:
results_neutral = pd.DataFrame(evaluate_model("NeuTraL", y_test_binary, y_preds_neutral_binary))
results_neutral

# GOAD

# RCA

In [13]:
from deepod.models.tabular import RCA

In [14]:
clf_rca = RCA(epochs=10)
clf_rca.fit(normal_data, y=None)
y_preds_rca = clf_rca.predict(X_test)

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=74, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=74, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals

100%|██████████| 10/10 [00:04<00:00,  2.49it/s]


In [15]:
y_preds_rca_binary = np.where(y_preds_rca == 0, 'normal', 'anomaly')

In [16]:
results_rca = pd.DataFrame(evaluate_model("RCA", y_test_binary, y_preds_rca_binary))
results_rca

,model,accuracy,precision,recall,f1
0,RCA,0.983509,1.0,0.894935,0.944555


# RDP

In [17]:
from deepod.models.tabular import RDP

In [18]:
clf_rdp = RDP(epochs=10)
clf_rdp.fit(normal_data, y=None)
y_preds_rdp = clf_rdp.predict(X_test)

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=74, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000006, time: 1.0s
epoch 10, training loss: 0.000009, time: 1.0s
Start Inference on the training data...


testing: 100%|██████████| 476/476 [00:00<00:00, 1113.61it/s]


In [19]:
y_preds_rdp_binary = np.where(y_preds_rdp == 0, 'normal', 'anomaly')

In [20]:
results_rdp = pd.DataFrame(evaluate_model("RDP", y_test_binary, y_preds_rdp_binary))
results_rdp

,model,accuracy,precision,recall,f1
0,RDP,0.965276,1.0,0.778778,0.875632


# REPEN

In [18]:
from deepod.models.tabular import REPEN

In [19]:
clf_repen = REPEN(epochs=1)
clf_repen.fit(normal_data, y=None)
y_preds_repdn = clf_repen.predict(X_test)

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=41, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)


KeyboardInterrupt: 

In [ ]:
y_preds_repen_binary = np.where(y_preds_repdn == 0, 'normal', 'anomaly')

In [ ]:
results_repen = pd.DataFrame(evaluate_model("REPEN", y_test_binary, y_preds_repen_binary))
results_repen

# Deep SVDD

In [21]:
from deepod.models.tabular import DeepSVDD

In [22]:
clf_deepsvdd = DeepSVDD(epochs=1)
clf_deepsvdd.fit(normal_data, y=None)
y_preds_deepsvdd = clf_deepsvdd.predict(X_test)

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=74, out_features=100, bias=False)
      (act_layer): ReLU()
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): ReLU()
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.020199, time: 0.4s
Start Inference on the training data...


testing: 100%|██████████| 476/476 [00:00<00:00, 2554.55it/s]


In [23]:
y_preds_deepsvdd_binary = np.where(y_preds_deepsvdd == 0, 'normal', 'anomaly')

In [24]:
results_deepsvdd = pd.DataFrame(evaluate_model("DeepSVDD", y_test_binary, y_preds_deepsvdd_binary))
results_deepsvdd

,model,accuracy,precision,recall,f1
0,DeepSVDD,0.983706,1.0,0.896191,0.945254


# Results

In [22]:
results_df = pd.concat([results_dif, results_slad], ignore_index=True)
results_df.to_csv('results/DeepOD_results.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,DIF,0.156965,0.156965,1.000000,0.271339
1,SLAD,0.983344,1.000000,0.893889,0.943972
